In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool

In [2]:
load_dotenv(override=True)

True

In [3]:
agent_instruction1 = "You are a sales agent working in ComplAI \
     a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI \
     You write professional, serious cold emails."

agent_instruction2 = "You are a sales agent working in ComplAI \
    a company that provides a Saas tool for ensurimg SOC2 compliance and preparing for audits, powered by AI \
    You write witty, engaging cold emails"

agent_instruction3 = "You are a sales agent working in ComplAI \
    a company that provides a Saas tool for ensurimg SOC2 compliance and preparing for audits, powered by AI \
    You write concise, to the point cold emails"

In [4]:
sales_agent1 = Agent(
    name = 'Professional sales agent',
    instructions = agent_instruction1,
    model = 'gpt-4o-mini'
)

sales_agent2 = Agent(
    name = 'Engaging sales agent',
    instructions = agent_instruction2,
    model = 'gpt-4o-mini'
)

sales_agent3 = Agent(
    name = 'Busy sales agent',
    instructions = agent_instruction3,
    model = 'gpt-4o-mini'
)

In [5]:
description = 'Write a cold sales email'
tool1 = sales_agent1.as_tool(tool_name='sales-agent-1', tool_description=description)
tool2 = sales_agent2.as_tool(tool_name='sales-agent-2', tool_description=description)
tool3 = sales_agent3.as_tool(tool_name='sales-agent-3', tool_description=description)

In [6]:
@function_tool
def save_to_html_file(content: str, filename: str = "output.html"):
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(content)
        print(f"Content successfully written to {filename}")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

In [7]:
save_to_html_file

FunctionTool(name='save_to_html_file', description='', params_json_schema={'properties': {'content': {'title': 'Content', 'type': 'string'}, 'filename': {'default': 'output.html', 'title': 'Filename', 'type': 'string'}}, 'required': ['content', 'filename'], 'title': 'save_to_html_file_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x12d218b80>, strict_json_schema=True)

In [8]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(
    name='Email subject writer',
    instructions=subject_instructions,
    model='gpt-4o-mini')

subject_tool = subject_writer.as_tool(tool_name='subject_writer', tool_description='Write a subject for a cold sales email')

html_converter = Agent(
    name='Html email body converter',
    instructions=html_instructions,
    model='gpt-4o-mini'
)

html_tool = html_converter.as_tool(tool_name='html_converter', tool_description='Convert a text email body to an HTML email body')

In [9]:
tools = [subject_tool, html_tool, save_to_html_file]

In [10]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x12d2194e0>, strict_json_schema=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x12d219620>, strict_json_schema=True),
 FunctionTool(name='save_to_html_file', description='', params_json_schema={'properties': {'content': {'title': 'Content', 'type': 'string'}, 'filename': {'d

In [11]:
instruction ="You are an email formatter and save the formatted email in a file. You receive the body of an email to be sent. \
    You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
    Finally, you use the save_to_html_file tool to save the email with the subject and HTML body in a file named 'output.html'"

emailer_agent = Agent(
    name='Email Manager',
    instructions=instruction,
    tools=tools,
    model='gpt-4o-mini',
    handoff_description='Convert an email to html and save it'
)

In [12]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [14]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and save the email."

sales_manager = Agent(
    name='Sales Manager',
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model='gpt-4o-mini'
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace('Automated SDR'):
    result = await Runner.run(sales_manager, message)

Content successfully written to output.html
